In [ ]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
!pip install -q git+https://github.com/huggingface/diffusers.git
!pip install -q transformers accelerate bitsandbytes sentencepiece hf_transfer qwen_vl_utils

In [ ]:
import torch
from diffusers import Flux2KleinPipeline
from huggingface_hub import login

login("")

In [ ]:
import torch
from diffusers import Flux2KleinPipeline
import ipywidgets as widgets
from IPython.display import display, clear_output

# MODEL
model_id = "black-forest-labs/FLUX.2-klein-4B"

pipe = Flux2KleinPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="balanced",
    low_cpu_mem_usage=True
)
pipe.enable_attention_slicing("max")

# STYLE
STYLE_PRESETS = {
    "realistic": {
        "suffix": ", ultra-realistic photography, 8k, shot on 35mm lens, cinematic lighting",
        "steps": 28,
        "guidance": 3.5
    },
    "cartoon": {
        "suffix": ", 3D cartoon style, vibrant colors, clean lines, Pixar inspired",
        "steps": 20,
        "guidance": 4.5
    },
    "ghibli": {
        "suffix": ", Studio Ghibli art style, hand-drawn, watercolor textures, soft lighting",
        "steps": 22,
        "guidance": 3.0
    },
    "neon_noir": {
        "suffix": ", cyberpunk noir, neon lights, rainy night, cinematic",
        "steps": 24,
        "guidance": 4.0
    },
    "oil_painting": {
        "suffix": ", classical oil painting, thick brush strokes, dramatic lighting",
        "steps": 30,
        "guidance": 4.0
    },
    "pixel_art": {
        "suffix": ", high-quality 16-bit pixel art, retro game aesthetic",
        "steps": 15,
        "guidance": 5.0
    }
}

# UI
prompt_input = widgets.Textarea(
    placeholder="Enter your prompt here...",
    description="Prompt:",
    layout=widgets.Layout(width="100%", height="80px")
)

style_dropdown = widgets.Dropdown(
    options=list(STYLE_PRESETS.keys()),
    value="ghibli",
    description="Style:"
)

generate_button = widgets.Button(
    description="Generate Image",
    button_style="success"
)

output = widgets.Output()

# GENERATION
def generate_image(_):
    with output:
        clear_output()

        base_prompt = prompt_input.value.strip()
        style_name = style_dropdown.value
        style = STYLE_PRESETS[style_name]

        if not base_prompt:
            print("Please enter a prompt.")
            return

        full_prompt = base_prompt + style["suffix"]

        print(f"Style selected: {style_name}")
        print("Generating image...")

        image = pipe(
            prompt=full_prompt,
            num_inference_steps=style["steps"],
            guidance_scale=style["guidance"],
            width=512,
            height=1112,
            generator=torch.manual_seed(42)
        ).images[0]

        filename = f"flux2_{style_name}.png"
        image.save(filename)

        display(image)
        print(f"Saved as {filename}")

generate_button.on_click(generate_image)

# DISPLAY
display(prompt_input, style_dropdown, generate_button, output)

# Watermark

In [ ]:
!pip install opencv-python pywavelets numpy

In [ ]:
import cv2
import numpy as np
import pywt
import hashlib
import os
import time

def embed_time_bound_watermark(
    img_path,
    secret_key,
    output_path,
    strength=0.06
):
    if not os.path.exists(img_path):
        raise FileNotFoundError("Image not found")

    img = cv2.imread(img_path)
    if img is None:
        raise ValueError("Failed to load image")

    # Current UNIX timestamp (seconds)
    timestamp = int(time.time())

    # Create payload
    payload = f"{secret_key}|{timestamp}"
    digest = hashlib.sha256(payload.encode()).digest()

    # Convert image
    ycrcb = cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)
    y, cr, cb = cv2.split(ycrcb)
    y = y.astype(np.float32) / 255.0

    # DWT
    LL, (LH, HL, HH) = pywt.dwt2(y, 'haar')

    # PN noise
    seed = int.from_bytes(digest[:4], 'big')
    rng = np.random.default_rng(seed)
    pn = rng.standard_normal(HL.shape)

    # Embed
    HL_marked = HL + strength * pn

    # Inverse DWT
    y_marked = pywt.idwt2((LL, (LH, HL_marked, HH)), 'haar')
    y_marked = np.clip(y_marked * 255, 0, 255).astype(np.uint8)

    watermarked = cv2.merge([y_marked, cr, cb])
    watermarked = cv2.cvtColor(watermarked, cv2.COLOR_YCrCb2BGR)

    cv2.imwrite(output_path, watermarked)

    print("watermark embedded")
    print("Creation timestamp :", timestamp)
    print(".................")

    return timestamp

In [ ]:
def detect_time_bound_watermark(
    img_path,
    secret_key,
    timestamp,
    threshold=0.08
):
    if not os.path.exists(img_path):
        raise FileNotFoundError("Image not found")

    img = cv2.imread(img_path)
    if img is None:
        raise ValueError("Failed to load image")

    y = cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)[:, :, 0]
    y = y.astype(np.float32) / 255.0

    # DWT
    LL, (LH, HL, HH) = pywt.dwt2(y, 'haar')

    # Recreate payload
    payload = f"{secret_key}|{timestamp}"
    digest = hashlib.sha256(payload.encode()).digest()

    seed = int.from_bytes(digest[:4], 'big')
    rng = np.random.default_rng(seed)
    pn = rng.standard_normal(HL.shape)

    hl_vec = HL.flatten()
    pn_vec = pn.flatten()

    score = np.dot(hl_vec, pn_vec) / (
        np.linalg.norm(hl_vec) * np.linalg.norm(pn_vec) + 1e-9
    )

    if score > threshold:
        watermark_id = hashlib.sha256(payload.encode()).hexdigest()[:16]
        print("WATERMARK VERIFIED")
        print("Watermark ID :", watermark_id)
        print("Created at   :", time.ctime(timestamp))
        print("Score        :", f"{score:.4f}")
        return True
    else:
        print("No valid watermark found")
        print("Score:", f"{score:.4f}")
        return False

In [ ]:
image_path = "/content/flux2_realistic.png"
output_path = "/content/watermarked_flux2_realistic.png"
secret_key = "MyUltraSecretKey123"

# Embed
creation_time = embed_time_bound_watermark(
    image_path,
    secret_key,
    output_path
)
# Detect later
detect_time_bound_watermark(
    output_path,
    secret_key,
    creation_time
)
